# Test results

With folder `data/cleaned/preds`, we can obtain the results for the position and error metrics but in an accumulated version and not as an average.

In [1]:
import glob, os, pandas as pd, numpy as np, torch, math
from tqdm import tqdm
from BronchoTrack import metrics
from torch import nn

def compute_metrics_acum(folder):
    aape, aade, aane, aace, aamse, aaqe, aamset = [], [], [], [], [], [], []
    for pred_path in tqdm(glob.glob(folder)):
        df = pd.read_csv(pred_path, index_col=0)
        acum = df.cumsum()
        pos_preds = torch.from_numpy(acum.loc[acum.index[-1], ["shift_x", "shift_y", "shift_z"]].values)
        pos_targets = torch.from_numpy(acum.loc[acum.index[-1], ["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
        aape.append(metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy())
        rot_targets = acum.loc[acum.index[-1], ["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
        rot_preds = acum.loc[acum.index[-1], ["Rx_dif", "Ry_dif", "Rz_dif"]].values
        rot_targets = torch.from_numpy(rot_targets)
        rot_preds = torch.from_numpy(rot_preds)
        aade.append(metrics.DirectionError.inverse_cos(rot_preds, rot_targets))
        aane.append(metrics.NeedleError.needle(torch.hstack([pos_preds, rot_preds]), torch.hstack([pos_targets, rot_targets]), distance=5))
        cse = metrics.CosMetric()
        cse.update(rot_preds, rot_targets)
        aace.append(cse.compute())
        q = metrics.QuatMetric()
        q.update(rot_preds, rot_targets)
        aaqe.append(q.compute())
        m = metrics.MSE()
        m.update(rot_preds, rot_targets)
        aamse.append(m.compute())
        ms = metrics.EuclideanDistance()
        ms.update(rot_preds, rot_targets)
        aamset.append(ms.compute())
    return "AAPE {} +- {}, AADE {} +- {}, AANE {} +- {} AACE {} +- {} AAMSE {} +- {} AAQE {} +- {} AAMSE2 {} +- {}".format(
        np.mean(aape), np.std(aape), np.mean(aade), np.std(aade), np.mean(aane), np.std(aane), np.mean(aace), np.std(aace),
        np.mean(aamse), np.std(aamse), np.mean(aaqe), np.std(aaqe), np.mean(aamset), np.std(aamset))

In [3]:
#compute_metrics_acum("data/cleaned/preds_30traj_MSE/*.csv"), compute_metrics_acum("data/cleaned/preds_30traj_COS/*.csv"), compute_metrics_acum("data/cleaned/preds_30traj_DE/*.csv"), compute_metrics_acum("data/cleaned/preds_30traj_QUAT/*.csv"), compute_metrics_acum("data/cleaned/preds_30traj_COS_EU/*.csv")
#folders = glob.glob("data/cleaned/all_outer/preds_outer_*/")
folders = glob.glob("data/cleaned/final_test/preds_*/")
for folder in folders:
    print(folder)
    print(compute_metrics_acum(folder + "*.csv"))
    print("\n\n")

data/cleaned/final_test/preds_offsetnet_loss_mse_ce_FINAL/


100%|██████████| 40/40 [00:00<00:00, 166.15it/s]


AAPE 5.694908300830166 +- 4.634883603055501, AADE 1.584564896136206 +- 0.6745711965221148, AANE 9.478710402793485 +- 4.165236212680044 AACE 0.9949752688407898 +- 0.3429638147354126 AAMSE 1248.64404296875 +- 4032.7666015625 AAQE 1.187364935874939 +- 0.31424471735954285 AAMSE2 38.993377685546875 +- 47.17466354370117



data/cleaned/final_test/preds_broncho_intra_model_convtemporal_loss_mse_ce_FINAL/


100%|██████████| 40/40 [00:00<00:00, 182.34it/s]

AAPE 4.994679840116618 +- 4.442020285378994, AADE 1.6334868961672278 +- 0.7475373035609071, AANE 9.3537357274942 +- 4.5839302353279265 AACE 0.8084970712661743 +- 0.35443809628486633 AAMSE 1048.977783203125 +- 3564.389892578125 AAQE 1.3135675191879272 +- 0.4368025064468384 AAMSE2 33.29506301879883 +- 45.14833068847656





In [2]:
def compute_metrics(folder):
    aape, aade, aane, aace, aamse, aaqe, aamset, aamset3 = [], [], [], [], [], [], [], []
    for pred_path in tqdm(glob.glob(folder)):
        df = pd.read_csv(pred_path, index_col=0)
        for row, data in df.iterrows():
            pos_preds = torch.from_numpy(data[["shift_x", "shift_y", "shift_z"]].values)
            pos_targets = torch.from_numpy(data[["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
            aape.append(metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy())
            rot_targets = data[["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
            rot_preds = data[["Rx_dif", "Ry_dif", "Rz_dif"]].values
            rot_targets = torch.from_numpy(rot_targets)
            rot_preds = torch.from_numpy(rot_preds)
            aade.append(metrics.DirectionError.inverse_cos(rot_preds, rot_targets))
            aane.append(metrics.NeedleError.needle(torch.hstack([pos_preds, rot_preds]), torch.hstack([pos_targets, rot_targets]), distance=3))
            cse = metrics.CosMetric()
            cse.update(rot_preds, rot_targets)
            aace.append(cse.compute())
            q = metrics.QuatMetric()
            q.update(rot_preds, rot_targets)
            aaqe.append(q.compute())
            m = metrics.MSE()
            m.update(rot_preds, rot_targets)
            aamse.append(m.compute())
            ms = metrics.EuclideanDistance()
            ms.update(rot_preds, rot_targets)
            aamset.append(ms.compute())
            m.reset(), ms.reset(), cse.reset() 
            ll = nn.MSELoss()
            aamset3.append(ll(rot_preds, rot_targets))
    return "AAPE {} +- {}, AADE {} +- {}, AANE {} +- {} AACE {} +- {} AAMSE {} +- {} AAQE {} +- {} AAMSE2 {} +- {} AAMSE3 {} +- {}".format(
        np.mean(aape), np.std(aape), np.mean(aade), np.std(aade), np.mean(aane), np.std(aane), np.mean(aace), np.std(aace),
        np.mean(aamse), np.std(aamse), np.mean(aaqe), np.std(aaqe), np.mean(aamset), np.std(aamset), np.mean(aamset3), np.std(aamset3) )

In [3]:
# compute_metrics("data/cleaned/data_increase/outer_120/preds_120traj_COS_EU_inter_P18/*.csv")
# compute_metrics("data/cleaned/architectures2/preds_15traj_COS_arch_doublelateconvtemporal2/*.csv")
# compute_metrics("data/cleaned/prune_distill/preds_15traj_COS_prune_06/*.csv")
# compute_metrics("data/cleaned/loss_intra/preds_30traj_DE/*.csv")
# compute_metrics("data/cleaned/comparison/preds_15traj_comparison_offsetnet/*.csv")
folders = glob.glob("data/cleaned/all_intra/preds_intra_*/")
for folder in folders:
    print(folder)
    print(compute_metrics(folder + "*.csv"))
    print("\n\n")

data/cleaned/all_intra/preds_intra_model_doublelateconvtemporal_loss_mse/


100%|██████████| 60/60 [00:10<00:00,  5.95it/s]


AAPE 0.3680805354096437 +- 0.24513293390149332, AADE 0.6552843961319902 +- 0.5022018649090123, AANE 1.892865354051268 +- 1.2457047195961222 AACE 0.15347787737846375 +- 0.1976008117198944 AAMSE 23.685781478881836 +- 705.59619140625 AAQE 0.6615489721298218 +- 0.3701571822166443 AAMSE2 1.1657220125198364 +- 8.34855842590332 AAMSE3 23.685781222474745 +- 705.5962464657475



data/cleaned/all_intra/preds_intra_model_doublelatetemporal_loss_cos/


100%|██████████| 60/60 [00:09<00:00,  6.07it/s]


AAPE 0.3711909002947466 +- 0.24085223694993516, AADE 0.7907587787835094 +- 0.5990097168044618, AANE 2.2127386338349195 +- 1.4268444582650024 AACE 0.21380412578582764 +- 0.2504816949367523 AAMSE 25.370847702026367 +- 703.092529296875 AAQE 0.7398276329040527 +- 0.4633330702781677 AAMSE2 1.8881011009216309 +- 8.517488479614258 AAMSE3 25.3708484049439 +- 703.0925312134837



data/cleaned/all_intra/preds_intra_model_doublelatetemporal_loss_direction/


...

AAPE 0.35535299998064485 +- 0.26564102929029265, AADE 0.7326312215530074 +- 0.5737959951085523, AANE 2.0508293808672677 +- 1.370302428861312 AACE 0.19561447203159332 +- 0.24284790456295013 AAMSE 25.289413452148438 +- 704.9070434570312 AAQE 0.7228149771690369 +- 0.4569717347621918 AAMSE2 1.8007925748825073 +- 8.522052764892578 AAMSE3 25.289414002665325 +- 704.9070279132632



data/cleaned/all_intra/preds_intra_model_doublelateconvtemporal_loss_direction/


100%|██████████| 60/60 [00:09<00:00,  6.11it/s]


AAPE 0.432721793142318 +- 0.2609068448908133, AADE 0.888110934984538 +- 0.6353997512902394, AANE 2.4398944305307992 +- 1.4807076083252955 AACE 0.4526693820953369 +- 0.3616095781326294 AAMSE 25.848838806152344 +- 703.1240844726562 AAQE 0.7656574845314026 +- 0.6099673509597778 AAMSE2 2.4807791709899902 +- 8.449394226074219 AAMSE3 25.84884098514213 +- 703.124145497759



data/cleaned/all_intra/preds_intra_model_doublelate_loss_cos/


100%|██████████| 60/60 [00:09<00:00,  6.11it/s]

AAPE 0.39534267490642483 +- 0.26850898332486245, AADE 0.735987522325995 +- 0.571709192510524, AANE 2.071517441883913 +- 1.3646237904923522 AACE 0.1886986345052719 +- 0.23782919347286224 AAMSE 25.15384292602539 +- 704.2326049804688 AAQE 0.7151168584823608 +- 0.45628446340560913 AAMSE2 1.7498983144760132 +- 8.508782386779785 AAMSE3 25.15384245578188 +- 704.2325937736316





In [18]:
compute_metrics("data/cleaned/preds_30traj_MSE/*.csv"), compute_metrics("data/cleaned/preds_30traj_COS/*.csv"), compute_metrics("data/cleaned/preds_30traj_DE/*.csv"), compute_metrics("data/cleaned/preds_30traj_QUAT/*.csv"), compute_metrics("data/cleaned/preds_30traj_COS_EU/*.csv")

100%|██████████| 120/120 [00:17<00:00,  6.93it/s]


('AAPE 0.5491277217666873 +- 0.573653173740771, AADE 0.7248276525217127 +- 0.498294865650828, AANE 2.133701451221059 +- 1.3643027063455693 AACE 0.17344273626804352 +- 0.19498062133789062 AAMSE 15.751633644104004 +- 455.8465576171875 AAQE 0.6379731297492981 +- 0.3691841959953308',
 'AAPE 0.2646646795385093 +- 0.16944264858997296, AADE 0.5428455738276546 +- 0.5032078981195843, AANE 1.5545040837767403 +- 1.2577643932778806 AACE 0.11609118431806564 +- 0.18177255988121033 AAMSE 25.304031372070312 +- 711.93359375 AAQE 0.6825854182243347 +- 0.4137953817844391',
 'AAPE 0.3682859682206188 +- 0.1913224449289966, AADE 0.713015618811532 +- 0.5331496554219437, AANE 2.0406247662187673 +- 1.3392663732322747 AACE 0.43466493487358093 +- 0.3853222131729126 AAMSE 26.017972946166992 +- 706.2825927734375 AAQE 0.749027669429779 +- 0.5646618604660034',
 'AAPE nan +- nan, AADE nan +- nan, AANE nan +- nan AACE nan +- nan AAMSE nan +- nan AAQE inf +- nan',
 'AAPE 0.25759985880518227 +- 0.16911057809014685, AADE

 ## Results loss cross patient

In [36]:
from collections import defaultdict

In [50]:
def compute_errors(root="./data/cleaned/preds_round_1/", losses=["COS", "MSE", "DE", "QUAT"]):
    subfolders = {i: [os.path.join(root, folder)  for folder in os.listdir(root) if i in folder] for i in losses}
    submetrics = {i: defaultdict(list) for i in losses}
    for loss in tqdm(losses):
        for subfolder in subfolders[loss]:
            for file in os.listdir(subfolder):
                df_sample = pd.read_csv(os.path.join(subfolder, file), index_col=0)
                for row, data in df_sample.iterrows():
                    pos_preds = torch.from_numpy(data[["shift_x", "shift_y", "shift_z"]].values)
                    pos_targets = torch.from_numpy(data[["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
                    submetrics[loss]["AAPE"].append(metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy())
                    rot_targets = data[["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
                    rot_preds = data[["Rx_dif", "Ry_dif", "Rz_dif"]].values
                    rot_targets = torch.from_numpy(rot_targets)
                    rot_preds = torch.from_numpy(rot_preds)
                    submetrics[loss]["AADE"].append(metrics.DirectionError.inverse_cos(rot_preds, rot_targets).numpy())
                    submetrics[loss]["AANE"].append(metrics.NeedleError.needle(torch.hstack([pos_preds, rot_preds]), torch.hstack([pos_targets, rot_targets]), distance=3).numpy())
        submetrics[loss]["AAPE"] = (np.mean(submetrics[loss]["AAPE"]), np.std(submetrics[loss]["AAPE"]))
        submetrics[loss]["AADE"] = (np.mean(submetrics[loss]["AADE"]), np.std(submetrics[loss]["AADE"]))
        submetrics[loss]["AANE"] = (np.mean(submetrics[loss]["AANE"]), np.std(submetrics[loss]["AANE"]))
    return submetrics
        


In [51]:
results = compute_errors()

  0%|          | 0/4 [00:00<?, ?it/s]

In [52]:
results

{'COS': defaultdict(list,
             {'AAPE': (0.7378152121278749, 0.4505294373549075),
              'AADE': (1.1975325795102736, 0.7669386701924008),
              'AANE': (3.253374431203741, 1.6608348329290958)}),
 'MSE': defaultdict(list,
             {'AAPE': (0.7555466617392615, 0.4499440452227794),
              'AADE': (1.2367228395662788, 0.7639783827566352),
              'AANE': (3.3043340700818193, 1.6125088852780254)}),
 'DE': defaultdict(list,
             {'AAPE': (0.7390488814078968, 0.4434353750507124),
              'AADE': (1.2119055731497668, 0.7702147501155583),
              'AANE': (3.2856565743090784, 1.666478388639561)}),
 'QUAT': defaultdict(list,
             {'AAPE': (0.7443752694683445, 0.44549288678195853),
              'AADE': (1.332005785650499, 0.6744398102944591),
              'AANE': (3.6061799354533117, 1.4897843823402674)})}

# Build docs for Debora

In [2]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
import torch
from BronchoTrack import metrics

In [8]:
def add_metrics(df_x):
    df_x["Direction Error"], df_x["Pos Error (Euclidean)"], df_x["Needle Error"], df_x["Cosinus Error"], df_x["Cosinus X Error"] = 0.0, 0.0, 0.0, 0.0, 0.0
    df_x["Cosinus Y Error"], df_x["Cosinus Z Error"], df_x["MSE"], df_x["Quaternion Error"] = 0.0, 0.0, 0.0, 0.0
    for row, data in df_x.iterrows():
            pos_preds = torch.from_numpy(data[["shift_x", "shift_y", "shift_z"]].values)
            pos_targets = torch.from_numpy(data[["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
            df_x.loc[row, "Pos Error"] = metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy()
            rot_targets = data[["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
            rot_preds = data[["Rx_dif", "Ry_dif", "Rz_dif"]].values
            rot_targets = torch.from_numpy(rot_targets)
            rot_preds = torch.from_numpy(rot_preds)
            df_x.loc[row, "Direction Error"] = metrics.DirectionError.inverse_cos(rot_preds, rot_targets).numpy()
            df_x.loc[row, "Needle Error"] = metrics.NeedleError.needle(torch.hstack([pos_preds, rot_preds]), torch.hstack([pos_targets, rot_targets]), distance=3).numpy()
            cse = metrics.CosMetric()
            cse.update(rot_preds, rot_targets)
            df_x.loc[row, "Cosinus Error"] = cse.compute().numpy()
            # q = metrics.QuatMetric()
            # q.update(rot_preds, rot_targets)
            # df_x.loc[row, "Quaternion Error "] = q.compute().numpy()
            m = metrics.MSE()
            m.update(rot_preds, rot_targets)
            df_x.loc[row, "MSE"] = m.compute().numpy()
            cse_x = metrics.CosMetric(0)
            cse_x.update(rot_preds, rot_targets)
            df_x.loc[row, "Cosinus X Error"] = cse_x.compute().numpy()
            cse_y = metrics.CosMetric(1)
            cse_y.update(rot_preds, rot_targets)
            df_x.loc[row, "Cosinus Y Error"] = cse_y.compute().numpy()
            cse_z = metrics.CosMetric(2)
            cse_z.update(rot_preds, rot_targets)
            df_x.loc[row, "Cosinus Z Error"] = cse_z.compute().numpy()
    return df_x


def add_info(df_x, filename, folder):
    df_x["Patient"] = filename.split("_")[1]
    df_x["Lobe"] = filename.split("_")[-2]
    df_x["Trajectory"] = filename.split("_")[-1].split(".")[0]
    df_x["Loss"] = folder.split("_")[-1]
    df_x["Architecture"] = folder.split("_")[-3]
    return df_x

In [11]:
root = "./data/cleaned/all_intra/"
final_csv = pd.DataFrame()
for folder in tqdm(os.listdir(root)):
    subfolder = os.path.join(root, folder)
    if os.path.isdir(subfolder) and "preds" in subfolder:
        for file in os.listdir(subfolder):
            df_sample = pd.read_csv(os.path.join(subfolder, file), index_col=0) 
            columns_to_drop = df_sample.columns
            df_sample = add_metrics(df_sample)
            df_sample = add_info(df_sample, file, folder)
            df_sample.drop(columns_to_drop, axis=1, inplace=True)
            if not final_csv.empty:
                final_csv = pd.concat([final_csv, df_sample], axis=0, ignore_index=True)
            else:
                final_csv = df_sample.copy()
final_csv.to_csv(os.path.join(root, "global_result_intra.csv"))

  0%|          | 0/16 [00:00<?, ?it/s]

## COmpute latencies

In [1]:
import torch
import time
from tqdm import tqdm_notebook
import numpy as np
from BronchoTrack.models import bronchonet
from BronchoTrack.models import offsetnet

In [2]:
model3d = bronchonet.BronchoNetDoubleLate3DFusion()
modelt = bronchonet.BronchoNetDoubleTemporalLateFusion()
model = bronchonet.BronchoNetDoubleLateFusion()
modelct = bronchonet.BronchoNetDoubleTemporalConvLateFusion()
modeloff = offsetnet.OffsetNet()

In [3]:
def compute_avg_latency(model):
    avg_t = []
    model.to("cuda:0")
    for i in tqdm_notebook(range(100)):
        t0 = time.time()
        out = model(torch.randn(1, 2, 3, 256, 256).to("cuda:0"))
        avg_t.append(time.time() -  t0)
    return np.mean(avg_t), np.std(avg_t)

In [4]:
compute_avg_latency(model)

ipykernel_launcher:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x16384 and 256x6)

In [26]:
compute_avg_latency(model)

/home/juan/miniconda3/envs/broncho/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/100 [00:00<?, ?it/s]

(0.028017663955688478, 0.002162735453398334)

In [27]:
compute_avg_latency(modelt)

/home/juan/miniconda3/envs/broncho/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/100 [00:00<?, ?it/s]

(0.028059961795806884, 0.0015364384959331034)

In [20]:
compute_avg_latency(modelct)

ipykernel_launcher:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/100 [00:00<?, ?it/s]

(0.03249024391174316, 0.05954236133072228)

In [3]:
sum([ p.numel() for p in modelct.parameters() if p.requires_grad])

14142398

In [6]:
from thop import profile

In [8]:
input = torch.randn(1, 2, 3, 224, 224).cuda()
macs, params = profile(modeloff, inputs=(input, ))

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


/home/juan/miniconda3/envs/broncho/lib/python3.6/site-packages/thop/vision/basic_hooks.py:116: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  [*(y.shape[2:])]


In [9]:
from thop import clever_format
macs, params = clever_format([macs, params], "%.3f")

In [10]:
macs, params

('14.685G', '43.607M')

# Cross subject acum

In [6]:
import glob, os, pandas as pd, numpy as np, torch, math
from tqdm import tqdm
from BronchoTrack import metrics
from torch import nn

def compute_metrics_acum(folders):
    aape, aade, aane, aace, aamse, aaqe, aamset = [], [], [], [], [], [], []
    for folder in folders:
        for pred_path in tqdm(glob.glob(folder + "*.csv")):
            df = pd.read_csv(pred_path, index_col=0)
            acum = df.cumsum()
            pos_preds = torch.from_numpy(acum.loc[acum.index[-1], ["shift_x", "shift_y", "shift_z"]].values)
            pos_targets = torch.from_numpy(acum.loc[acum.index[-1], ["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
            aape.append(metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy())
            rot_targets = acum.loc[acum.index[-1], ["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
            rot_preds = acum.loc[acum.index[-1], ["Rx_dif", "Ry_dif", "Rz_dif"]].values
            rot_targets = torch.from_numpy(rot_targets)
            rot_preds = torch.from_numpy(rot_preds)
            aade.append(metrics.DirectionError.inverse_cos(rot_preds, rot_targets))
            aane.append(metrics.NeedleError.needle(torch.hstack([pos_preds, rot_preds]), torch.hstack([pos_targets, rot_targets]), distance=5))
            cse = metrics.CosMetric()
            cse.update(rot_preds, rot_targets)
            aace.append(cse.compute())
            q = metrics.QuatMetric()
            q.update(rot_preds, rot_targets)
            aaqe.append(q.compute())
            m = metrics.MSE()
            m.update(rot_preds, rot_targets)
            aamse.append(m.compute())
            ms = metrics.EuclideanDistance()
            ms.update(rot_preds, rot_targets)
            aamset.append(ms.compute())
    return "AAPE {} +- {}, AADE {} +- {}, AANE {} +- {} AACE {} +- {} AAMSE {} +- {} AAQE {} +- {} AAMSE2 {} +- {}".format(
        np.mean(aape), np.std(aape), np.mean(aade), np.std(aade), np.mean(aane), np.std(aane), np.mean(aace), np.std(aace),
        np.mean(aamse), np.std(aamse), np.mean(aaqe), np.std(aaqe), np.mean(aamset), np.std(aamset))

In [10]:
folders = glob.glob("data/cleaned/all_outer/preds_outer_*_doublelate3d_*/")
print(folders)
print(compute_metrics_acum(folders))
print("\n\n")

['data/cleaned/all_outer/preds_outer_P20_model_doublelate3d_loss_mse/', 'data/cleaned/all_outer/preds_outer_P30_model_doublelate3d_loss_mse/', 'data/cleaned/all_outer/preds_outer_P18_model_doublelate3d_loss_mse/', 'data/cleaned/all_outer/preds_outer_P21_model_doublelate3d_loss_mse/', 'data/cleaned/all_outer/preds_outer_P25_model_doublelate3d_loss_mse/']


100%|██████████| 60/60 [00:00<00:00, 183.53it/s]

AAPE 18.245185363661033 +- 8.158070239513572, AADE 1.5837502060292499 +- 0.7343845994917435, AANE 19.2106833783095 +- 8.468442244945606 AACE 0.7175427079200745 +- 0.35592326521873474 AAMSE 1112.3828125 +- 4590.41455078125 AAQE 1.357682228088379 +- 0.4473152756690979 AAMSE2 24.104469299316406 +- 52.49878692626953





# Cross subject mean

In [11]:
def compute_metrics(folder):
    aape, aade, aane, aace, aamse, aaqe, aamset, aamset3 = [], [], [], [], [], [], [], []
    for folder in folders:
        for pred_path in tqdm(glob.glob(folder+ "*.csv")):
            df = pd.read_csv(pred_path, index_col=0)
            for row, data in df.iterrows():
                pos_preds = torch.from_numpy(data[["shift_x", "shift_y", "shift_z"]].values)
                pos_targets = torch.from_numpy(data[["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
                aape.append(metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy())
                rot_targets = data[["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
                rot_preds = data[["Rx_dif", "Ry_dif", "Rz_dif"]].values
                rot_targets = torch.from_numpy(rot_targets)
                rot_preds = torch.from_numpy(rot_preds)
                aade.append(metrics.DirectionError.inverse_cos(rot_preds, rot_targets))
                aane.append(metrics.NeedleError.needle(torch.hstack([pos_preds, rot_preds]), torch.hstack([pos_targets, rot_targets]), distance=3))
                cse = metrics.CosMetric()
                cse.update(rot_preds, rot_targets)
                aace.append(cse.compute())
                q = metrics.QuatMetric()
                q.update(rot_preds, rot_targets)
                aaqe.append(q.compute())
                m = metrics.MSE()
                m.update(rot_preds, rot_targets)
                aamse.append(m.compute())
                ms = metrics.EuclideanDistance()
                ms.update(rot_preds, rot_targets)
                aamset.append(ms.compute())
                m.reset(), ms.reset(), cse.reset() 
                ll = nn.MSELoss()
                aamset3.append(ll(rot_preds, rot_targets))
    return "AAPE {} +- {}, AADE {} +- {}, AANE {} +- {} AACE {} +- {} AAMSE {} +- {} AAQE {} +- {} AAMSE2 {} +- {} AAMSE3 {} +- {}".format(
        np.mean(aape), np.std(aape), np.mean(aade), np.std(aade), np.mean(aane), np.std(aane), np.mean(aace), np.std(aace),
        np.mean(aamse), np.std(aamse), np.mean(aaqe), np.std(aaqe), np.mean(aamset), np.std(aamset), np.mean(aamset3), np.std(aamset3) )

In [14]:
folders = glob.glob("data/cleaned/all_outer/preds_outer_*_doublelate3d_*/")

print(folders)
print(compute_metrics(folders))
print("\n\n")

['data/cleaned/all_outer/preds_outer_P20_model_doublelate3d_loss_mse/', 'data/cleaned/all_outer/preds_outer_P30_model_doublelate3d_loss_mse/', 'data/cleaned/all_outer/preds_outer_P18_model_doublelate3d_loss_mse/', 'data/cleaned/all_outer/preds_outer_P21_model_doublelate3d_loss_mse/', 'data/cleaned/all_outer/preds_outer_P25_model_doublelate3d_loss_mse/']


100%|██████████| 60/60 [00:07<00:00,  8.53it/s]


AAPE 0.7114064937362017 +- 0.3393738549947833, AADE 0.7399739287277504 +- 0.547025555708217, AANE 2.212906160303557 +- 1.3519748053809486 AACE 0.1724420189857483 +- 0.20338161289691925 AAMSE 33.27992630004883 +- 1081.2686767578125 AAQE 0.6641477942466736 +- 0.39331161975860596 AAMSE2 1.3146730661392212 +- 9.905120849609375 AAMSE3 33.279925538216354 +- 1081.2687085392918





# Compute direction error on indexes

In [17]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import torch
from BronchoTrack import metrics
import numpy as np

In [20]:
def add_metrics(df_x):
    df_x["Direction Error"], df_x["Direction Error X"], df_x["Direction Error Y"], df_x["Direction Error Z"] = 0.0, 0.0, 0.0, 0.0
    for row, data in df_x.iterrows():
            rot_targets = data[["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
            rot_preds = data[["Rx_dif", "Ry_dif", "Rz_dif"]].values
            rot_targets = torch.from_numpy(rot_targets)
            rot_preds = torch.from_numpy(rot_preds)
            df_x.loc[row, "Direction Error X"] = metrics.DirectionError.inverse_cos(rot_preds, rot_targets).numpy()
            df_x.loc[row, "Direction Error Y"] = metrics.DirectionError.inverse_cos(rot_preds, rot_targets, axis=1).numpy()
            df_x.loc[row, "Direction Error Z"] = metrics.DirectionError.inverse_cos(rot_preds, rot_targets, axis=2).numpy()
            df_x.loc[row, "Direction Error"] = np.mean([df_x.loc[row, "Direction Error Z"], df_x.loc[row, "Direction Error X"], df_x.loc[row, "Direction Error Y"]])
    return df_x

def add_info(df_x, filename, folder):
    df_x["Patient"] = filename.split("_")[1]
    df_x["Lobe"] = filename.split("_")[-2]
    df_x["Trajectory"] = filename.split("_")[-1].split(".")[0]
    df_x["Loss"] = folder.split("_")[-1]
    df_x["Architecture"] = folder.split("_")[-3]
    return df_x

In [21]:
root = "./data/cleaned/final_test/"
dest = "./data/cleaned/final_test/"
final_csv = pd.DataFrame()
for folder in tqdm(os.listdir(root)):
    subfolder = os.path.join(root, folder)
    if os.path.isdir(subfolder) and "preds" in subfolder:
        name = "bronchonet" if "broncho" in folder else "offsetnet"
        dest_folder = os.path.join(root, name)
        os.makedirs(dest_folder, exist_ok=True)
        for file in os.listdir(subfolder):
            df_sample = pd.read_csv(os.path.join(subfolder, file), index_col=0) 
            df_sample = add_metrics(df_sample)
            df_sample = add_info(df_sample, file, folder)
            final_csv = df_sample.to_csv(os.path.join(dest_folder, file))

/home/juan/miniconda3/envs/broncho/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/6 [00:00<?, ?it/s]